# Evaluación de los algoritmos de agrupar

Tenemos 3 propuestas, donde cada una tiene un total de 10 clusters: la primera hace los grupos de acuerdo a las palabras clave de los subtítulos, la segunda hace lo mismo pero por vídeos, ambas utilizando el algoritmo de k-means, mientras que la última lo hace por videos pero utilizando el espectro y k-mean

In [41]:
import pickle
import json
#Para el corpus
import sys
sys.path.append('..')
from src.data import Data 
#Para calcular TF-IDF
from src.tfidf import TFIDF
from tqdm import tqdm
from operator import itemgetter
# nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords #Listas de stopwords
from collections import Counter

[nltk_data] Downloading package punkt to /home/karla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Y si agrupamos por el título? cómo vectorizamos el título? se puede igual con tf.idf?

### Cargamos los clusters de cada intento

In [4]:
def load(path):
    """
    Carga clusters, regresa los id y documentos
    de cada cluster

    Args:
        path (str): ruta del archivo
    """
    with open(path, "rb") as f:
        model = pickle.load(f)
    return model[0], model[1]

In [5]:
#Por subtítulo
named_subKM, clusters_subKM = load("../pkl/sub_classKMeans.pkl")
#Por vídeo utilizando espectro
named_videoKM, clusters_videoKM = load("../pkl/video_classKMeans.pkl")
#Por vídeo utilizando espectro y k-means
named_videoS, clusters_videoS = load("../pkl/video_classSpectre.pkl")

#### Intento 1

De los canales de los que se tomaron los videos sabemos que uno de ellos es de comida y otro trata temas políticos, por lo que como una primera prueba podemos ver si los vídeos de esos canales quedaron en los mismos clusters (sería de alguna manera verificar la pureza de 2 clusters)

Obtenemos los id de los videos de los canales mencionados:

|           id            |    canal      |
|-------------------------|---------------|
|UCJQQVLyM6wtPleV4wFBK06g | visualpolitik |
|UC_Zc2fmbDpu_arkwvCDcX5g | cocina        |

In [6]:
clusters_goal = {'cocina':[], 'política': []}
files = [('cocina','UC_Zc2fmbDpu_arkwvCDcX5g'), ('política','UCJQQVLyM6wtPleV4wFBK06g')]
for name, file in files:
    with open("../corpus/data/"+file+".json", 'rb') as f:
        chanel = json.loads(f.read().decode('utf-8', 'replace'))
        for video in chanel:
            clusters_goal[name].append(video["id"])

In [7]:
print(clusters_goal['cocina'][:10])

['IAHQXsLS3P8', 'dhYWMdHVvGg', 'ULle2vGte8k', 'gun3ouyEoIc', 'H2L_btIXzzc', 'r5IM3yYTQoU', 'ld2pqKmEdDg', 'aNEHF2g0Bos', 'GbQOvcv1cTU', 'DUhvKm4VLeg']


In [8]:
def check_goal_clusters(named_video):
    """
    Muestra en pantalla la distribución de los vídeos contenidos
    en los clusters meta
    
    Args:
        named_video (dict): diccionario con el nombre del clusters y el id de los videos
    """
    resp = [[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]]
    for i, name in enumerate(clusters_goal):
        for id in clusters_goal[name]:
            for c, docs in named_video.items():
                if id in docs:
                    resp[i][c] += 1
    print("""Distribución de videos en los clusters
         Cocina:
            {}
        Política:
            {}""".format(resp[0], resp[1]))

Lo intentamos para los videos utilizando k-means

In [9]:
check_goal_clusters(named_videoKM)


Distribución de videos en los clusters
         Cocina:
            [40, 0, 129, 0, 1, 74, 7, 0, 0, 1]
        Política:
            [1, 0, 13, 0, 1, 694, 0, 0, 1, 0]


In [10]:
len(named_videoKM[5])

1698

Con lo anterior podemos tomar que el cluster 2 es el que corresponde a Cocina y que el cluster 5 a política, aunque el cluster 5 cuenta con 1698 videos de los que 694 son de política y 74 de cocina, lo que deja 930 vídeos que podrían ser de política o podrían ser un error al momento de agrupar

Lo intentamos ahora con los vídeos utilizando el espectro

In [11]:
check_goal_clusters(named_videoS)

Distribución de videos en los clusters
         Cocina:
            [29, 14, 24, 20, 4, 24, 31, 22, 84, 0]
        Política:
            [52, 44, 63, 50, 31, 45, 75, 31, 319, 0]


Notemos que aquí los videos de ambas categorías están distribuidos a lo largo de todos los clusters

#### Intento 2

Luego podemos extraer las principales palabras de los clusters para ver si tienen relación y de verdad representan un grupo

In [12]:
#Cargamos el corpus limpio
videos = Data("../pkl/clean_videos.pkl")
videos.get_all_subtitles()
#Creamos el objeto tfidf
tfidf_obj = TFIDF(videos)
# Cargamos los valores tfidf previamente calculados
tfidf_obj.tfidf = "../pkl/tfidf.pkl" #Vectores por texto

100%|██████████| 9/9 [00:00<00:00, 12.83it/s]


In [37]:
more = ["si", "bien", "ahora", "así", "aquí", "pues"]
stopwords_list = stopwords.words('spanish') + more

def take2(tfidf):
    for w, _ in tfidf:
        if w not in stopwords_list:
            return w

In [61]:
#Cargamos tfidf por documento y obtenemos la palabra clave de cada uno
#las agrupamos por clusters
def get_keywords_clusters(named_video):
    '''
    Regresa un diccionario con los subtítulos y su vector tfidf
    '''
    
    keyword = [[],[],[],[],[],[],[],[],[],[]]
    #Iteramos sobre los canales
    for c, docs in named_video.items():
        for id in docs:
            key = take2(sorted(tfidf_obj.tfidf[id].items(), key=itemgetter(1), reverse=True))
            keyword[c].append(key)
        keyword[c] = sorted(Counter(keyword[c]).items(), key=itemgetter(1), reverse=True)[:5]
    return keyword


In [62]:
keyword_kmeans = get_keywords_clusters(named_videoKM)
for i, val in enumerate(keyword_kmeans):
    print("Cluster: {}\n Palabras clave: {}\n----------------".format(i, val))

Cluster: 0
 Palabras clave: [('música', 39), ('años', 38), ('voy', 30), ('vamos', 22), ('vídeo', 15)]
----------------
Cluster: 1
 Palabras clave: [('música', 20), ('voy', 10), ('mundo', 6), ('años', 6), ('vamos', 5)]
----------------
Cluster: 2
 Palabras clave: [('música', 140), ('voy', 44), ('vídeo', 38), ('años', 35), ('país', 23)]
----------------
Cluster: 3
 Palabras clave: [('música', 13), ('vamos', 7), ('voy', 7), ('país', 5), ('años', 4)]
----------------
Cluster: 4
 Palabras clave: [('música', 17), ('voy', 16), ('años', 11), ('vamos', 10), ('amigos', 9)]
----------------
Cluster: 5
 Palabras clave: [('música', 629), ('país', 530), ('años', 163), ('vídeo', 107), ('voy', 43)]
----------------
Cluster: 6
 Palabras clave: [('música', 38), ('voy', 28), ('años', 25), ('vamos', 22), ('país', 8)]
----------------
Cluster: 7
 Palabras clave: [('música', 4), ('hola', 2), ('anime', 2), ('aplausos', 1), ('vamos', 1)]
----------------
Cluster: 8
 Palabras clave: [('música', 47), ('voy', 23

Algo salió mal porque todos los clusters tienen a música como la palabra representativa

In [63]:
keyword_S = get_keywords_clusters(named_videoS)
for i, val in enumerate(keyword_S):
    print("Cluster: {}\n Palabras clave: {}\n----------------".format(i, val))

Cluster: 0
 Palabras clave: [('música', 97), ('país', 42), ('vídeo', 32), ('años', 25), ('voy', 23)]
----------------
Cluster: 1
 Palabras clave: [('música', 59), ('país', 26), ('años', 8), ('vídeo', 5), ('voy', 5)]
----------------
Cluster: 2
 Palabras clave: [('música', 71), ('país', 42), ('vídeo', 15), ('vamos', 11), ('años', 11)]
----------------
Cluster: 3
 Palabras clave: [('música', 52), ('país', 34), ('voy', 14), ('vídeo', 10), ('años', 6)]
----------------
Cluster: 4
 Palabras clave: [('música', 42), ('país', 22), ('años', 9), ('gobierno', 4), ('voy', 4)]
----------------
Cluster: 5
 Palabras clave: [('música', 62), ('país', 45), ('años', 30), ('vamos', 15), ('voy', 11)]
----------------
Cluster: 6
 Palabras clave: [('música', 103), ('país', 59), ('años', 25), ('vamos', 23), ('vídeo', 20)]
----------------
Cluster: 7
 Palabras clave: [('música', 103), ('país', 55), ('años', 40), ('vídeo', 18), ('voy', 18)]
----------------
Cluster: 8
 Palabras clave: [('música', 399), ('país',

#### Intento 3

También podemos comparar entre los diferentes clusters de cada intento para ver si coinciden algunos

In [78]:
#Hacemos una intersección de clusters
inter = [0,0,0,0,0,0,0,0,0,0]
for c, docs in named_videoKM.items():
    for id in docs:
        if id in named_videoS[c]:
            inter[c] += 1
    inter[c] = {'C': c, 
                'Total video KM': len(docs),
                'Total video S': len(named_videoS[c]),
                'Compartidos': inter[c]}

In [79]:
inter

[{'C': 0, 'Total video KM': 219, 'Total video S': 296, 'Compartidos': 17},
 {'C': 1, 'Total video KM': 84, 'Total video S': 134, 'Compartidos': 2},
 {'C': 2, 'Total video KM': 376, 'Total video S': 192, 'Compartidos': 25},
 {'C': 3, 'Total video KM': 61, 'Total video S': 143, 'Compartidos': 1},
 {'C': 4, 'Total video KM': 98, 'Total video S': 103, 'Compartidos': 1},
 {'C': 5, 'Total video KM': 1698, 'Total video S': 210, 'Compartidos': 140},
 {'C': 6, 'Total video KM': 188, 'Total video S': 313, 'Compartidos': 20},
 {'C': 7, 'Total video KM': 23, 'Total video S': 302, 'Compartidos': 2},
 {'C': 8, 'Total video KM': 230, 'Total video S': 1471, 'Compartidos': 136},
 {'C': 9, 'Total video KM': 299, 'Total video S': 112, 'Compartidos': 13}]